In [ ]:
import numpy as np
import sklearn as sk
from sklearn.datasets import load_iris
iris=load_iris()

# Shuffling the dataset randomly
perm=np.random.permutation(len(iris.data))
x_iris=iris.data[perm]
y_iris=iris.target[perm]

#Splitting manually into train and test set
x_iris_train=x_iris[:120]
y_iris_train=y_iris[:120]
x_iris_test=x_iris[120:]
y_iris_test=y_iris[120:]

predicted_classes = np.empty((len(x_iris_test))) #Initialising an empty numpy array of size=number of test samples

for n, sample in enumerate(x_iris_test):  #traverse along with indexes through test set
  q = np.empty((len(x_iris_train), 2))   #an array q of rows=number of train samples to store distance from each train sample
  for x, train_sample in enumerate(x_iris_train):
    #calculating "distance"
    p = sample - train_sample
    s = np.dot(p, p)
    #store result in q
    q[x] = [s, y_iris_train[x]]
  #sorting data column wise
  sorted_indices = np.argsort(q[:, 0]) #returns indices that would sort the array with respect to column 0
  sorted_q = q[sorted_indices]
  a = sorted_q[0:5] #get first 5 elements : 5NN
  j = a[:, 1] #get classes of those neighbours
  #now simply count number of occurences of each class in J and classify sample to majority
  count0 = 0
  count1 = 0
  count2 = 0
  for k in j:
    if k == 0:  count0 += 1
    elif k == 1:  count1 += 1
    else:   count2 += 1
  if count0 > count1 and count0 > count2:
    predicted_classes[n] = 0
  elif count1 > count0 and count1 > count2:
    predicted_classes[n] = 1
  else:
    predicted_classes[n] = 2

#print the whole predicted class array
print("Predicted classes array:", predicted_classes)

#Analysis of this result using 3 class confusion matrix
print("Analysing our model.")

def calculate_confusion_matrix(y_true, y_pred, num_classes):
  # Initialize the confusion matrix of size(nXn) where n=number of classes
  conf_matrix = np.zeros((num_classes, num_classes))
  # Iterate over each sample to create confusion matrix with respect to A
  #.     A.   B.   C.   Predicted
  #.  A  TP   FN   FN
  #   B  FP   TN   TN
  #   C  FP   TN   TN
  for true_class, pred_class in zip(y_true, y_pred):
    #Update the corresponding cell in the confusion matrix
    conf_matrix[int(true_class), int(pred_class)] += 1
  return conf_matrix
def calculate_metrics(conf_matrix):
  #Calculating true positives, false positives, false negatives for each class
  TP = np.diag(conf_matrix)  #obtained as a matrix of order 1xn
  FP = np.sum(conf_matrix, axis=0) - TP   #sum of column-TP
  FN = np.sum(conf_matrix, axis=1) - TP   #sum of row -TP
  # calculating true negatives for each class
  TN = np.sum(conf_matrix) - (TP + FP + FN) #remaining part

  # calculating accuracy for each class
  accuracy = (TP + TN) / np.sum(conf_matrix)

  # calculating precision for each class
  precision = TP / (TP + FP)

  # calculating recall for each class
  recall = TP / (TP + FN)

  # calculating F1 score for each class
  f1_score = 2 * (precision * recall) / (precision + recall)

  return accuracy, precision, recall, f1_score


conf_matrix=calculate_confusion_matrix(y_iris_test,predicted_classes,3)
accuracy, precision, recall, f1_score = calculate_metrics(conf_matrix)
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Predicted classes array: [1. 0. 1. 1. 0. 2. 2. 1. 0. 2. 2. 2. 1. 0. 0. 2. 0. 0. 0. 1. 1. 2. 2. 0.
 1. 0. 0. 0. 1. 2.]
Analysing our model.
[[12.  0.  0.]
 [ 0.  9.  2.]
 [ 0.  0.  7.]]
Accuracy: [1.         0.93333333 0.93333333]
Precision: [1.         1.         0.77777778]
Recall: [1.         0.81818182 1.        ]
F1 Score: [1.    0.9   0.875]
